In [24]:
import pandas as pd
import numpy as np
import os
import xml.etree.ElementTree as ET
import cv2
import json

# inits - populated by the user
folder = FOLDER_OF_IMAGES
vocs = FOLDER_OF_VOC_LABELS
ids = TXT_FILE_OF_IDS
save_folder = PATH_TO_FOLDER_TO_SAVE_COCO_ANNOTATIONS
temp = []
with open(train_ids) as f:
    for line in f:
        temp.append(line.strip())
all_ids = temp

out = { # dictionary in coco format
    'images': [],
    'type': "instances",
    'annotations': [],
    'categories': [{'supercategory': 'none', 'id': 1, 'name': 'logo'}]
}

# set images dictionary format
for ID in all_ids:
    put = {
        'file_name': ID + ".png", 'id': ID,
        'height': 1440, 'width': 2560,
    }
    out['images'].append(put)
    
def get_bbox(bbox):
    # convert VOC to COCO bounding box
    b = []
    for i in range(4):
        b.append(int(bbox[i].text))
    x1, y1, x2, y2 = b
    w = x2 - x1
    h = y2 - y1
    b, area = [x1, y2, w, h], w*h
    return(b, area)
           
i = 1
for ID in all_ids:
    xml = vocs + ID + ".xml"
    root = ET.parse(xml).getroot()
    bboxs = root.findall('./object/bndbox') # find all bounding boxes for this image ID
    for bbox in bboxs:
        # convert each bounding box to coco format and store
        b, a = get_bbox(bbox)
        put = {
            'area' : a,'bbox': b, 'image_id': ID, 'id': i,
            'iscrowd': 0, 'ignore': 0, 'segmentation': [], 'category_id': 1
        }
        out['annotations'].append(put)
        i += 1
        
with open(save_folder + '/coco_labels.json', 'w') as outfile:
    json.dump(out, outfile) # write coco file